In [1]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 

In [2]:
os.listdir("Data/")

['.ipynb_checkpoints',
 'IMDB Movie Dataset Info.docx',
 'movie-model.png',
 'movie.sql',
 'movieERD.mwb',
 'movietestkey.sql',
 'title-akas-us-only.csv',
 'title.basics.cleaned.csv',
 'title.basics.tsv.gz',
 'title.ratings.cleaned.csv',
 'title.ratings.tsv.gz']

## 1) Start by loading the title-akas-us-only.csv file. All non-us movies have been removed from this file for you.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
fname= 'Data/title-akas-us-only.csv'
akas = pd.read_csv(fname, low_memory=False)

In [4]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
2,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
3,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
4,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


In [5]:
akas.shape

(1452564, 8)

## 2) Load the title basics file next.

In [6]:
basics = pd.read_csv('Data/title.basics.tsv.gz', sep='\t', low_memory=False)

In [7]:
basics.shape

(10017011, 9)

In [8]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


## 3) Remove non-US movies from title basics


In [9]:
# Filter the basics table down to only include the US by using the filter akas dataframe
filter_us_titles = basics['tconst'].isin(akas['titleId'])
basics = basics[filter_us_titles]
basics.shape

(1365643, 9)

## 4) Perform the remaining cleaning steps for title basics.

In [10]:
# Convert placeholder "\N" values back to true null values to identify missing genres and runtimes.
akas = akas.replace({'\\N':np.nan})
basics = basics.replace({'\\N':np.nan})

In [11]:
# Drop rows with null values in the runtimeMinutes or genres columns ONLY.
basics = basics.dropna(subset=["runtimeMinutes","genres"])

In [12]:
# Confirming no more null
basics['genres'].isna().sum()

0

In [13]:
basics['titleType'].unique()

array(['short', 'movie', 'tvSeries', 'tvEpisode', 'tvMovie', 'tvShort',
       'tvMiniSeries', 'video', 'tvSpecial', 'videoGame'], dtype=object)

In [14]:
# Filter to keep only full-length movies (titleType==Movie)
filter = basics['titleType']=='movie'
basics = basics[filter]
basics.shape

(203476, 9)

In [15]:
# Convert startYear to a float dtype
basics['startYear'] = basics['startYear'].astype(float)
basics['startYear'].dtype

dtype('float64')

In [16]:
# Filter to keep movies with startYears that are >=2000 and <=2022
filter = (basics['startYear']>=2000) & (basics['startYear']<=2022)
basics = basics[filter]
basics.shape

(121127, 9)

In [17]:
# Eliminate movies that include "Documentary" in genre
filter = basics['genres'].str.contains('Documentary')
# Exclude movies in the documentary category
basics = basics[~filter]

## 5) Display a final preview of your filtered title basics and save to a csv

In [18]:
basics.tail()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
10016149,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
10016544,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
10016684,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
10016693,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"
10016777,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History"


In [19]:
basics['startYear'] = basics['startYear'].astype(int)
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86979 entries, 34802 to 10016777
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          86979 non-null  object
 1   titleType       86979 non-null  object
 2   primaryTitle    86979 non-null  object
 3   originalTitle   86979 non-null  object
 4   isAdult         86979 non-null  object
 5   startYear       86979 non-null  int32 
 6   endYear         0 non-null      object
 7   runtimeMinutes  86979 non-null  object
 8   genres          86979 non-null  object
dtypes: int32(1), object(8)
memory usage: 6.3+ MB


In [20]:
# Save the data to a CSV file in your Data folder.
fname_out = 'Data/title.basics.cleaned.csv'
basics.to_csv(fname_out, index=False)

In [21]:
os.listdir("Data/")

['.ipynb_checkpoints',
 'IMDB Movie Dataset Info.docx',
 'movie-model.png',
 'movie.sql',
 'movieERD.mwb',
 'movietestkey.sql',
 'title-akas-us-only.csv',
 'title.basics.cleaned.csv',
 'title.basics.tsv.gz',
 'title.ratings.cleaned.csv',
 'title.ratings.tsv.gz']

## 6) Load and filter the title ratings file

In [22]:
ratings = pd.read_csv('Data/title.ratings.tsv.gz', sep='\t', low_memory=False)
filter_basics = ratings['tconst'].isin(basics['tconst'])
ratings = ratings[filter_basics]

In [23]:
ratings.head()

,tconst,averageRating,numVotes
17961,tt0035423,6.4,87153
40764,tt0062336,6.4,175
46645,tt0069049,6.7,7754
63640,tt0088751,5.2,336
69953,tt0096056,5.6,846


In [24]:
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

## 7) Display a final preview of your filtered title ratings and save to a csv

In [25]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71900 entries, 17961 to 1331462
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         71900 non-null  object 
 1   averageRating  71900 non-null  float64
 2   numVotes       71900 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [26]:
# Save the data to a CSV file in your Data folder.
fname_out = 'Data/title.ratings.cleaned.csv'
ratings.to_csv(fname_out, index=False)